<a href="https://colab.research.google.com/github/gijigae/gradio/blob/master/Whisper_YouTube_in_SRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title INSTALL DEPENDENCIES
!pip install --quiet ffmpeg-python git+https://github.com/openai/whisper.git > /dev/null --q
!pip3 install gradio --q
!pip install pytube
!pip3 install vtt_to_srt3 --q

In [ ]:
#@title Launch the APP

import gradio as gr
from pytube import YouTube
import os
import subprocess

from vtt_to_srt.vtt_to_srt import vtt_to_srt

def transcribe_youtube(source_language, model_size, youtube_link, source_file=None):
    if(os.path.exists('audio.m4a')):
      os.remove('audio.m4a')
      os.remove('audio.m4a.vtt')
      os.remove('audio.m4a.srt')
    
    audio = "audio.m4a"

    if youtube_link != "":
      yt = YouTube(youtube_link)
      yt.streams.filter(only_audio=True)[0].download(filename=audio)
      subprocess.run('whisper audio.m4a --language ' + str(source_language) + ' --model ' + str(model_size), shell=True)
    
    # A logic to process an uploaded audio file
    elif source_file is not None:
      subprocess.run('whisper ' + source_file + ' --language ' + str(source_language) + ' --model ' + str(model_size), shell=True)
      audio = source_file.split("/")[2]
    else:
        return "You must either provide a YouTube link or a source file"
    
    path = '/content/' + audio + '.vtt'
    vtt_to_srt(path)
    with open('/content/' + audio + '.srt','r') as file:
      data=file.read()
    return data,'/content/' + audio + '.srt'

demo = gr.Interface(
    fn=transcribe_youtube,
    inputs=[
        gr.Textbox(lines=1, placeholder="enter the source language (Japanese, English, etc)"),
        gr.Dropdown(label="Model Size", choices=['base','small', 'medium', 'large'], value='small'),
        gr.Textbox(lines=2, placeholder="paste youtube link here", optional=True),
        # A gradio inteface to upload an audio file
        gr.Audio(source="upload", type="filepath", optional=True)
    ],
    outputs=[gr.Textbox(label="Transcripts"),gr.File(label="Download Transcripts")],
    allow_flagging="never"
)
demo.launch(debug=True)